In [3]:
import re
from AdvancedCalculator.utils import is_float

In [4]:
def is_last_num_or_bracket(text: str) ->bool:
    result = re.search("[0-9\)]", text[-1])
    if result is None:
        return False
    else:
        return True 
def are_parentheses_matched(text: str) -> bool:
    result = 0
    for element in text:
        if element == "(":
            result += 1
        elif element == ")":
            result -= 1
        if(result < 0):
            return False
    if result != 0:
        return False
    return True
def is_only_floats(list_of_str: list) -> bool:
    for element in list_of_str:
        if not element:
            continue
        elif is_float(element):
            continue
        else:
            return False
    return True
def is_enable_chars(text: str) -> bool:
    pattern = r'[\+\-\*/\(\)]'
    results = re.split(pattern, text)
    return is_only_floats(results)
def is_formula(text: str) -> bool:
    return is_last_num_or_bracket(text) and are_parentheses_matched(text) and is_enable_chars(text)

In [5]:
testText = "4(3+2/3"
is_formula(testText)

False

In [6]:
result

NameError: name 'result' is not defined

In [8]:
results

['3 ', ' 2', '10 ', '1', '30', '']

In [9]:
from sympy import sympify


In [81]:
from AdvancedCalculator.basicOperations import add, sub, mul, div
from AdvancedCalculator.utils import try_get_float_value

def get_id_of_open_bracket(list_of_math_sign :list) -> int:
    for index, element in enumerate(list_of_math_sign):
        if element == "(":
            return index
    return None
def get_id_of_close_bracket(list_of_math_sign: list) -> int:
    count = 0
    for index, element in enumerate(list_of_math_sign):
        if element == "(": 
            count += 1
        elif element == ")":
            count -= 1
        if count == 0:
            return index 
    return None

def try_get_list_of_brackets(list_of_math_elements: list) -> list:
    list_of_brackets = []

    begin_id = get_id_of_open_bracket(list_of_math_elements)
    if begin_id is None:
        return list_of_brackets
    end_id = begin_id + 1 + get_id_of_close_bracket(list_of_math_elements[begin_id:])
    if end_id is None:
        return list_of_brackets
    
    result = (begin_id, end_id)
    list_of_brackets.append(result)
    results = try_get_list_of_brackets(list_of_math_elements[end_id:])
    for result in results:
        result = (result[0]+ end_id, result[1] + end_id)
        list_of_brackets.append(result)
    return list_of_brackets

def calculate_formula(list_of_math_elements: list) -> list:

    list_of_brackets = try_get_list_of_brackets(list_of_math_elements)


    results = []
    for brackets in list_of_brackets:
        result = calculate_formula(list_of_math_elements[brackets[0]+1:brackets[1]-1])
        results = results + result
        
    for result, brackets in zip(reversed(results), reversed(list_of_brackets)):
        del list_of_math_elements[brackets[0]:brackets[1]]
        list_of_math_elements.insert(brackets[0], "*")
        list_of_math_elements.insert(brackets[0]+1, result)
        
    index = 0
    while index < len(list_of_math_elements):
        if list_of_math_elements[index] == "*":
            value1 = float(list_of_math_elements[index-1])
            value2 = float(list_of_math_elements[index+1])
            result = mul(value1, value2)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.insert(index - 1, str(result))
        elif list_of_math_elements[index] == "/":        
            value1 = float(list_of_math_elements[index-1])
            value2 = float(list_of_math_elements[index+1])
            result = div(value1, value2)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.insert(index - 1, str(result))
        else:
            index +=1   

        
    index = 0
    while index < len(list_of_math_elements):
        if list_of_math_elements[index] == "+":
            value1 = float(list_of_math_elements[index-1])
            value2 = float(list_of_math_elements[index+1])
            result = add(value1, value2)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.insert(index - 1, str(result))
        elif list_of_math_elements[index] == "-":        
            value1 = float(list_of_math_elements[index-1])
            value2 = float(list_of_math_elements[index+1])
            result = sub(value1, value2)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.pop(index -1)
            list_of_math_elements.insert(index - 1, str(result))
        else:
            index +=1  

    return list_of_math_elements

def convert_to_list(text_formula: str) -> list:
    pattern = r'\d+|[\+\-\*/\(\)0-9]'
    list_of_math_elements = re.findall(pattern, testText)
    return list_of_math_elements

In [82]:
testText = "3+2(10 -1* 30) + 1(2 - 3*4)" # -37
list_of_math_elements = convert_to_list(testText)
result = calculate_formula(list_of_math_elements)
print(result)

['-47.0']
